In [1]:
from sys import argv

import numpy as np
import pandas as pd 
from io_helper import read_tsp, normalize
from neuron import generate_network, get_neighborhood, get_route, get_drone_route
from distance import select_closest, euclidean_distance, route_distance
from plot import plot_network, plot_route

# def main():
#     if len(argv) != 2:
#         print("Correct use: python src/main.py <filename>.tsp")
#         return -1

#     problem = read_tsp(argv[1])

#     route = som(problem, 100000)

#     problem = problem.reindex(route)

#     distance = route_distance(problem)

#     print('Route found of length {}'.format(distance))


In [2]:
qa_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/qa194.tsp'
ca_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/ca4663.tsp'
eg_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/eg7146.tsp'

it_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/it16862.tsp'
fi_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/fi10639.tsp'
mo_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/mo14185.tsp'
pm_path = '/Users/gadmohamed/Desktop/mac_local/som-tsp/assets/pm8079.tsp'

qa_problem = read_tsp(qa_path)
ca_problem = read_tsp(ca_path)
eg_problem = read_tsp(eg_path)
it_problem = read_tsp(it_path)
fi_problem = read_tsp(fi_path)
mo_problem = read_tsp(mo_path)
pm_problem = read_tsp(pm_path)


Problem with 194 cities read.
Problem with 4663 cities read.
Problem with 7146 cities read.
Problem with 16862 cities read.
Problem with 10639 cities read.
Problem with 14185 cities read.
Problem with 8079 cities read.


In [4]:
def my_som(problem, iterations, learning_rate=0.8, population_size=8):
    nodes = problem.copy()
    # normalize the nodes coordinates
    nodes[['x', 'y']] = normalize(nodes[['x', 'y']])
    n = nodes.shape[0] * population_size
    network = generate_network(n)

    # learning_rate = 0.8
    learning_rate_decay = 0.99997
    nh_radius = n//10
    nh_radius_decay = 0.9997

    for i in range(iterations):
        # select a random node
        random_node = nodes.sample(1)[['x', 'y']].values

        # find the winner
        winner_idx = select_closest(network, random_node)

        # get the gaussian weighted neighborhood
        gaussian = get_neighborhood(winner_idx, nh_radius, n) 

        # update 
        network += gaussian[:, np.newaxis] * learning_rate * (random_node - network)

        # decay the learning rate and the gaussian radius
        learning_rate = learning_rate * learning_rate_decay
        nh_radius = nh_radius * nh_radius_decay
    plot_network(nodes, network, name = 'mine.png')
    route = get_route(nodes, network)
    return route 


def specific_som(problem, iterations, learning_rate=0.8, population_size=8):
    nodes = problem.copy()
    # normalize the nodes coordinates
    nodes[['x', 'y']] = normalize(nodes[['x', 'y']])
    n = nodes.shape[0] * population_size
    network = generate_network(n)

    iteartions = 100
    # learning_rate = 0.8
    learning_rate_decay = 0.99997
    nh_radius = n//10
    nh_radius_decay = 0.9997

    for i in range(iterations):
        # select a random node
        random_node = nodes.sample(1)[['x', 'y']].values

        # find the winner
        winner_idx = select_closest(network, random_node)

        # get the gaussian weighted neighborhood
        gaussian = get_neighborhood(winner_idx, nh_radius, n) 

        # update 
        network += gaussian[:, np.newaxis] * learning_rate * (random_node - network)

        # decay the learning rate and the gaussian radius
        learning_rate = learning_rate * learning_rate_decay
        nh_radius = nh_radius * nh_radius_decay
        # Check if any parameter has completely decayed.
        if nh_radius < 1:
            print('Radius has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
        if learning_rate < 0.001:
            print('Learning rate has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
    plot_network(nodes, network, name = 'mine.png')
    route = get_drone_route(nodes, network)
    reversed_route = route[::-1]
    concat_route = np.concatenate((route, reversed_route))
    
    return concat_route 



In [11]:
# time 
import time
problems = [it_problem, fi_problem, mo_problem, pm_problem]
problem_names = ['Italy', 'Finland', 'Moldova', 'Panama']
iterationss = [100, 1_000, 10_000]
results = pd.DataFrame(columns=['problem', 'iterations', 'time', 'distance'])
drone_results = pd.DataFrame(columns=['problem', 'iterations', 'time', 'distance'])
j = 0
for i in range(len(problems)):
    for iterations in iterationss:
        start = time.time()
        route = my_som(problems[i], iterations)
        end = time.time()
        start2 = time.time()
        drone_route = specific_som(problems[i], iterations)
        end2 = time.time()

        solution = qa_problem.reindex(route)
        distance = route_distance(solution)

        drone_solution = qa_problem.reindex(drone_route)
        drone_distance = route_distance(drone_solution)
        
        results.loc[j] = [problem_names[i], iterations, end-start, distance]
        drone_results.loc[j] = [problem_names[i], iterations, end2-start2, drone_distance]
        j += 1
        break 
    break 


In [6]:
results.head(20)

,problem,iterations,time,distance
0,Italy,100,42.755904,NaN
1,Italy,1000,46.940730,NaN
2,Italy,10000,106.599956,NaN
3,Finland,100,15.924031,NaN
4,Finland,1000,19.866709,NaN
5,Finland,10000,54.597417,NaN
6,Moldova,100,27.478717,NaN
7,Moldova,1000,32.427479,NaN
8,Moldova,10000,79.073409,NaN
9,Panama,100,8.920180,NaN


In [7]:
drone_results

,problem,iterations,time,distance
0,Italy,100,42.154917,NaN
1,Italy,1000,46.637094,NaN
2,Italy,10000,103.000843,NaN
3,Finland,100,15.885557,NaN
4,Finland,1000,19.358218,NaN
5,Finland,10000,54.059328,NaN
6,Moldova,100,27.691360,NaN
7,Moldova,1000,31.857430,NaN
8,Moldova,10000,78.199921,NaN
9,Panama,100,8.602560,NaN


In [26]:
sample_problem = read_tsp(qa_path)
route = my_som(sample_problem, 1000)
drone_route = specific_som(sample_problem, 1000)

solution = sample_problem.reindex(route)
drone_solution = sample_problem.reindex(drone_route)

distance = route_distance(solution)
drone_distance = route_distance(drone_solution)

route.shape , drone_route.shape, distance, drone_distance

Problem with 194 cities read.


((194,), (386,), 20398.47830648382, 32981.21015589726)

In [29]:
solution.columns

Index(['city', 'y', 'x'], dtype='object')

In [9]:
orig_sol = qa_problem.reindex(qa_route)
drone_sol = qa_problem.reindex(qa_route2)

orig_sol.shape,    drone_sol.shape

((194, 3), (386, 3))

In [10]:
orig_distance = route_distance(orig_sol)
drone_distance = route_distance(drone_sol)
orig_distance, drone_distance

(9972.728916793381, 19070.68764821045)